# API Documentation - Interactive Testing

This notebook provides interactive testing capabilities for the API endpoints. Run the setup cell first, then execute individual endpoint tests as needed.

In [ ]:
# Setup - Install required packages and import libraries
import requests
import json
from datetime import datetime
import os

# Configuration
BASE_URL = "http://localhost:3000"
session_token = None  # Will be set after login

def make_request(method, endpoint, data=None, cookies=None, params=None):
    """Helper function to make API requests"""
    url = f"{BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}
    
    try:
        if method.upper() == "GET":
            response = requests.get(url, headers=headers, cookies=cookies, params=params)
        elif method.upper() == "POST":
            response = requests.post(url, headers=headers, json=data, cookies=cookies)
        
        print(f"\n{method.upper()} {endpoint}")
        print(f"Status Code: {response.status_code}")
        print(f"Response: {json.dumps(response.json(), indent=2)}")
        
        return response
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None
    except json.JSONDecodeError:
        print(f"Status Code: {response.status_code}")
        print(f"Response: {response.text}")
        return response

print("Setup complete! Ready to test API endpoints.")

## Authentication Endpoints

### Register a New User

In [ ]:
# POST /api/auth/register
register_data = {
    "email": "test@example.com",
    "password": "testpassword123",
    "name": "Test User"
}

response = make_request("POST", "/api/auth/register", data=register_data)

# Store the token if registration successful
if response and response.status_code == 200:
    result = response.json()
    if "token" in result:
        session_token = result["token"]
        print(f"\nSession token stored: {session_token[:20]}...")

### Login

In [ ]:
# POST /api/auth/login
login_data = {
    "email": "test@example.com",
    "password": "testpassword123"
}

response = make_request("POST", "/api/auth/login", data=login_data)

# Store the token if login successful
if response and response.status_code == 200:
    result = response.json()
    if "token" in result:
        session_token = result["token"]
        print(f"\nSession token updated: {session_token[:20]}...")

### Get Current User Info

In [ ]:
# GET /api/auth/me
if session_token:
    cookies = {"session-token": session_token}
    make_request("GET", "/api/auth/me", cookies=cookies)
else:
    print("No session token available. Please login first.")

### Logout

In [ ]:
# POST /api/auth/logout
if session_token:
    cookies = {"session-token": session_token}
    response = make_request("POST", "/api/auth/logout", cookies=cookies)
    
    if response and response.status_code == 200:
        session_token = None
        print("\nSession token cleared.")
else:
    print("No session token available.")

## Lead Management Endpoints

### Enrich Company Data

In [ ]:
# POST /api/leads/enrich
enrich_data = {
    "company": "Anthropic",
    "website": "anthropic.com"
}

make_request("POST", "/api/leads/enrich", data=enrich_data)

### Score a Lead

In [ ]:
# POST /api/leads/score
score_data = {
    "leadId": "lead_123",
    "industry": "AI/ML",
    "company": "Anthropic",
    "employees": 500,
    "revenue": 100000000
}

make_request("POST", "/api/leads/score", data=score_data)

### Get Leads (Authenticated)

In [ ]:
# GET /api/leads
if session_token:
    cookies = {"session-token": session_token}
    params = {
        "page": 1,
        "limit": 10,
        "industry": "SaaS"
    }
    make_request("GET", "/api/leads", cookies=cookies, params=params)
else:
    print("Authentication required. Please login first.")

### Scrape Leads

In [ ]:
# POST /api/leads
scrape_data = {
    "searchParams": {
        "industry": "Technology",
        "location": "San Francisco",
        "companySize": "50-200"
    }
}

make_request("POST", "/api/leads", data=scrape_data)

## Analytics Endpoints

### Get Real-time Analytics

In [ ]:
# GET /api/analytics/realtime
if session_token:
    cookies = {"session-token": session_token}
    params = {"timeRange": "24h"}
    make_request("GET", "/api/analytics/realtime", cookies=cookies, params=params)
else:
    print("Authentication required. Please login first.")

## Admin Endpoints (Requires Admin Role)

### Get All Users

In [ ]:
# GET /api/admin/users
if session_token:
    cookies = {"session-token": session_token}
    make_request("GET", "/api/admin/users", cookies=cookies)
else:
    print("Authentication required. Please login first.")

### Create New User (Admin)

In [ ]:
# POST /api/admin/users
if session_token:
    cookies = {"session-token": session_token}
    new_user_data = {
        "email": "newuser@example.com",
        "password": "password123",
        "name": "New User",
        "role": "user"
    }
    make_request("POST", "/api/admin/users", data=new_user_data, cookies=cookies)
else:
    print("Authentication required. Please login first.")

## Enrichment Endpoints

### General Enrichment

In [ ]:
# POST /api/enrichment
enrichment_data = {
    "company": "OpenAI",
    "website": "openai.com"
}

make_request("POST", "/api/enrichment", data=enrichment_data)

## Utility Functions

In [ ]:
# Utility functions for testing

def test_all_auth_endpoints():
    """Test all authentication endpoints in sequence"""
    global session_token
    
    print("=== Testing Authentication Flow ===")
    
    # Register
    register_data = {
        "email": f"testuser_{datetime.now().strftime('%Y%m%d_%H%M%S')}@example.com",
        "password": "testpassword123",
        "name": "Test User"
    }
    
    response = make_request("POST", "/api/auth/register", data=register_data)
    if response and response.status_code == 200:
        result = response.json()
        if "token" in result:
            session_token = result["token"]
    
    # Get user info
    if session_token:
        cookies = {"session-token": session_token}
        make_request("GET", "/api/auth/me", cookies=cookies)
    
    print("\n=== Authentication Flow Complete ===")

def test_all_lead_endpoints():
    """Test all lead management endpoints"""
    print("\n=== Testing Lead Management Endpoints ===")
    
    # Enrich
    enrich_data = {"company": "Stripe", "website": "stripe.com"}
    make_request("POST", "/api/leads/enrich", data=enrich_data)
    
    # Score
    score_data = {"leadId": "lead_456", "industry": "FinTech"}
    make_request("POST", "/api/leads/score", data=score_data)
    
    print("\n=== Lead Management Testing Complete ===")

# Run quick test
def quick_test():
    """Run a quick test of key endpoints"""
    test_all_auth_endpoints()
    test_all_lead_endpoints()

print("Utility functions loaded. Use quick_test() to run a comprehensive test.")

## WebSocket Testing

In [ ]:
# WebSocket connection testing
# Note: This requires additional libraries for WebSocket testing
try:
    import websocket
    import threading
    
    def on_message(ws, message):
        print(f"WebSocket message received: {message}")
    
    def on_error(ws, error):
        print(f"WebSocket error: {error}")
    
    def on_close(ws, close_status_code, close_msg):
        print("WebSocket connection closed")
    
    def on_open(ws):
        print("WebSocket connection opened")
        # Send a test message
        ws.send("Hello WebSocket!")
    
    def test_websocket():
        if session_token:
            ws_url = f"ws://localhost:3000/api/websocket"
            headers = {"Cookie": f"session-token={session_token}"}
            
            ws = websocket.WebSocketApp(ws_url,
                                      on_open=on_open,
                                      on_message=on_message,
                                      on_error=on_error,
                                      on_close=on_close,
                                      header=headers)
            
            # Run in a separate thread
            ws_thread = threading.Thread(target=ws.run_forever)
            ws_thread.daemon = True
            ws_thread.start()
            
            print("WebSocket test started. Check console for messages.")
            return ws
        else:
            print("Authentication required for WebSocket connection.")
    
    print("WebSocket testing functions loaded. Use test_websocket() to test.")
    
except ImportError:
    print("websocket-client not installed. Install with: pip install websocket-client")
    
    def test_websocket():
        print("WebSocket testing requires 'websocket-client' package.")
        print("Install with: pip install websocket-client")

## Status Check

In [ ]:
# Check current session status
print(f"Base URL: {BASE_URL}")
print(f"Session Token: {'Set' if session_token else 'Not set'}")
if session_token:
    print(f"Token preview: {session_token[:20]}...")

# Test server connectivity
try:
    response = requests.get(f"{BASE_URL}/api/auth/me")
    print(f"\nServer connectivity: OK (Status: {response.status_code})")
except requests.exceptions.RequestException as e:
    print(f"\nServer connectivity: FAILED - {e}")
    print("Make sure your API server is running on localhost:3000")